# Building a ANN using PyTorch

# Import Libraries

In [76]:
import pandas as pd
import numpy as np

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Import Data

In [79]:
df = pd.read_csv('fmnist_small.csv')
df.shape

(6000, 785)

In [80]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


# Extract Features and Labels

In [81]:
X = df.drop('label', axis=1)
X = X.values
y = df['label'].values

In [82]:
X[0][100:150]

array([206, 200, 205, 208, 208, 216, 213, 244,  88,   0,   3,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   4,   0,   6,
       213, 218, 201, 197, 207, 212, 209, 219, 220, 240,  77,   0,   3,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

# Scale Data

In [83]:
X = X/255.0
X[0][100:150]

array([0.80784314, 0.78431373, 0.80392157, 0.81568627, 0.81568627,
       0.84705882, 0.83529412, 0.95686275, 0.34509804, 0.        ,
       0.01176471, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01568627, 0.        ,
       0.02352941, 0.83529412, 0.85490196, 0.78823529, 0.77254902,
       0.81176471, 0.83137255, 0.81960784, 0.85882353, 0.8627451 ,
       0.94117647, 0.30196078, 0.        , 0.01176471, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

# Train Test Split

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

X_train.shape, X_test.shape

((4800, 784), (1200, 784))

# `Dataset` Class

In [85]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.from_numpy(features).float()
    self.labels = torch.from_numpy(labels).long()

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, index):

    return self.features[index], self.labels[index]


In [86]:
train_dataset = CustomDataset(features=X_train, labels=y_train)
test_dataset = CustomDataset(features=X_test, labels=y_test)

In [87]:
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = 32,
    shuffle= True
  )

In [88]:
test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = 32,
    shuffle= True
  )

# Custom Model

In [89]:
class MyANN(nn.Module):
  def __init__(self, num_features):

    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(in_features=num_features, out_features=128),
        nn.ReLU(),
        nn.Linear(in_features=128, out_features=64),
        nn.ReLU(),
        nn.Linear(in_features=64, out_features=10)
    )

  def forward(self, features):
    return self.model(features)

## Set Learning Rate and Epochs

In [90]:
learning_rate = 0.01
epochs = 25

In [91]:
# Model Creation
model = MyANN(X_train.shape[1])

# Initiate Loss Function
ce_loss = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

## Training Pipeline

In [92]:
for epoch in range(epochs):
  losses = []

  for batch_features, batch_labels in  train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = ce_loss(y_pred, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update params
    optimizer.step()

    # store losses
    losses.append(loss.item())

  avg_loss = np.mean(losses)
  print(f'Epochs: {epoch+1}, Loss: {avg_loss}')

Epochs: 1, Loss: 2.2527875582377117
Epochs: 2, Loss: 2.027312146027883
Epochs: 3, Loss: 1.5910456482569377
Epochs: 4, Loss: 1.2359409948190054
Epochs: 5, Loss: 1.0426842693487803
Epochs: 6, Loss: 0.9269856230417888
Epochs: 7, Loss: 0.8565222771962484
Epochs: 8, Loss: 0.8060518131653468
Epochs: 9, Loss: 0.7723949654897054
Epochs: 10, Loss: 0.7433504150311152
Epochs: 11, Loss: 0.7173804414272308
Epochs: 12, Loss: 0.696763146718343
Epochs: 13, Loss: 0.6767992248137792
Epochs: 14, Loss: 0.6570929563045502
Epochs: 15, Loss: 0.6390217796961466
Epochs: 16, Loss: 0.6246746327479681
Epochs: 17, Loss: 0.6128489065170288
Epochs: 18, Loss: 0.5980106381575266
Epochs: 19, Loss: 0.5878133304913838
Epochs: 20, Loss: 0.5786966761946678
Epochs: 21, Loss: 0.5667369045813878
Epochs: 22, Loss: 0.5599262076616287
Epochs: 23, Loss: 0.5516477478543917
Epochs: 24, Loss: 0.5434744268655777
Epochs: 25, Loss: 0.5324187527100245


# Evaluation

## Set `eval` Model (Crucial)

In [93]:
model.eval()

MyANN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

## Eval Code

In [94]:
total_rows = 0
correct = 0

with torch.no_grad():
  for features, labels in test_loader:

      # Output from network: [batch, 10]
      y_pred = model(features)

      # Predicted class index
      _, predict = torch.max(y_pred, 1)

      # Count correct predictions
      correct += (predict == labels).sum().item()
      total_rows += labels.size(0)

  avg_accuracy = correct / total_rows
  print(f'Average Accuracy: {avg_accuracy:.4f}')


Average Accuracy: 0.8075
